In [ ]:
using Revise

In [ ]:
function hasduplicates(s::T where {T <: Union{String, Array}})::Bool 
    d = Dict{eltype(s), Int}()
    for v in s
        d[v] = get!(d, v, 0) + 1
    end
    return maximum(values(d)) > 1
end

println(hasduplicates("sssa"))
println(hasduplicates([1,2,33,3,3,44]))

In [ ]:
function bdayparadox(numstudents::Int)::Float64
    return 1 - ((365 -1)/365)^(0.5numstudents*(numstudents-1))
end

println(bdayparadox(23))

In [ ]:
using Distributed
@everywhere using SharedArrays

println("Threads: ",Threads.nthreads())
println("Procs: $(nprocs()), Workers: $(nworkers())")

In [ ]:
@everywhere function bdayrand(numstudents, numtrials)
    seq = @distributed (+) for i = 1:numtrials
       length(Set(rand(1:365, 23))) < numstudents ? 1 : 0
    end
    return seq/numtrials
end

@time bdayrand(23, 1000000)

In [ ]:
@code_native bdayrand(23, 1000)

In [ ]:
 function showtypetree(T, level=0)
     println("\t" ^ level, T)
     for t in subtypes(T)
         if t != Any
             showtypetree(t, level+1)
         end
    end
 end
 
 showtypetree(Number)

In [ ]:
abstract type MyType <: Real end
supertype(MyType)

mutable struct myint <: MyType
       val::Int16
end

foo = myint(23)
foo.val

In [ ]:
@everywhere struct randfield
    hasdupes::AbstractFloat
    function randfield(rrange::UnitRange{Int}, size::Int, samplesize::Int)::AbstractFloat
        u::Int = length(Set(rand(rrange, size)))
        x::AbstractFloat = size > u ? 1.0/samplesize : 0.0
        new(x)
    end
end

# test
randvars = randfield(1:365,23, 1000)
randvars.hasdupes

In [ ]:
@everywhere function cbdayrand(numstudents::Int, numtrials::Int)::Float64
    seq::Float64 = 0.0
    seq = @distributed (+) for i = 1:numtrials
       randfield(1:365, numstudents, numtrials).hasdupes
    end
    return seq
end
@time cbdayrand(23, 1000000)

### Interpretaion 1:
* Words have letters removed in a strict order; either removing the starting and ending letters or removing middle letters with the starting letters, and the resulting children are then recursively checked for validity.

In [ ]:
@everywhere struct wordmemo
    words::Vector{String}
    @inline function wordmemo()
        words = ["a", "i", ""]
        append!(words, readwords())
        new(words)
    end
end

@everywhere function Base.:in(str::String, wm::wordmemo)::Bool
    return str in wm.words::Vector{String}
end

@everywhere knownreductions1 = Dict{String, Vector{String}}()

@everywhere @inline splitmid(str::String)::String = join(split(str, str[sum(divrem(length(str), 2))]))

@everywhere function trimstring(str::String)::Vector{String}
    get!(knownreductions, str) do
        if length(str) == 1 || isempty(str)
            return [str]
        elseif length(str)%2 == 0
            seq = [str[2:end], str[1:end-1]]
        else
            seq = [str[2:end], splitmid(str), str[1:end-1]]
        end
    end
end


@everywhere function recurse1(str::String, memo::wordmemo)::Bool
    if str in memo
        if length(str) == 1 || length(str) == 0
            return true
        else
            return all([recurse(substr, memo) for substr in trimstring(str)])
        end
    else
        return false
    end
end

@inline function recurse_all1(words::Vector{String}, base::wordmemo)::Vector{String}
    return filter(x -> recurse(x, base), words)
end

@everywhere words = readwords()
@everywhere basememo = wordmemo()

@time recurse_all1(words, basememo)

In [ ]:
include("anagrams.jl")

In [45]:
using JLD2, FileIO

JLDBFILENAME = ""::String
const JLDBKEY = "ANAGRAMS"

function readwords()::Vector{String}
    wordslist = []
    for line in eachline("words.txt")
        push!(wordslist, line)
    end
    return wordslist
end

function get_anagrams(wordslist::Vector{String})::Dict{String, Vector{String}}
    wordsset = Dict{String, Vector{String}}()

    for word in wordslist
        key = join(sort([l for l in word]))
        wordsset[key] = push!(get!(wordsset, key, []), word)
    end
    return filter!(kv -> length(kv[2]) >= 2, wordsset)
end

function JLDSAVE(filename::String, dict::Dict{String, Vector{String}})
    global JLDBKEY
    save(filename, Dict(JLDBKEY=>dict))
end

function JLDLOAD(filename::String)
    global JLDBKEY
    return load(filename)[JLDBKEY]
end

function storeanagrams(name::String)
    global JLDBFILENAME
    basewords::Vector{String} = readwords()
    anagramsbook::Dict{String, Vector{String}} = get_anagrams(basewords)
    name = name*".jld2"
    try
        JLDSAVE(name, anagramsbook)
        JLDBFILENAME = name
    catch exc
        println("(!) Error: $exc")
    end
end

function readanagrams(word::String)::Vector{String}
    RET::Vector{String} = []
    word = join(sort!([w for w in word]))
    global JLDBFILENAME
    try
        if isfile(JLDBFILENAME)
            RET = JLDLOAD(JLDBFILENAME)[word]
        else
            error("(!) Error: DB File Doesn't Exist, create one by calling storeanagrams(filename)")
        end
    catch exc
        println("(!) Error: $exc")
    finally
        return RET
    end
end

readanagrams (generic function with 1 method)

In [48]:
storeanagrams("booyah")

"booyah.jld2"

In [49]:
readanagrams("boost")

2-element Array{String,1}:
 "boost"
 "boots"